In [ ]:
import os
from zipfile import ZipFile
import numpy as np
import pandas as pd
from collections import Counter
import math
import nltk
import string
from collections import defaultdict
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
with ZipFile('large_movie_review_dataset.zip','r') as zipOb:
  zipOb.extractall('dataset')

In [ ]:
root = 'dataset'
reviews = [] # document text
classifier = [] #positve or negative
for subdir, dirs, files in os.walk(root):
  for file in files:
    f = open(os.path.join(subdir,file), 'r')
    txt = f.read()
    if (subdir.endswith('pos')):
      classifier.append('pos')
    elif (subdir.endswith('neg')):
      classifier.append('neg')
    reviews.append(txt) #remove_stopwords(txt)  
print (reviews[3],"\n",classifier[3])

I have copy of this on VHS, I think they (The television networks) should play this every year for the next twenty years. So that we don't forget what was and that we remember not to do the same mistakes again. Like putting some people in the director's chair, where they don't belong. This movie Rappin' is like a vaudevillian musical, for those who can't sing, or act. This movie is as much fun as trying to teach the 'blind' to drive a city bus.<br /><br />John Hood, (Peebles) has just got out of prison and he's headed back to the old neighborhood. In serving time for an all-to-nice crime of necessity, of course. John heads back onto the old street and is greeted by kids dogs old ladies and his peer homeys as they dance and sing all along the way.<br /><br />I would recommend this if I was sentimental, or if in truth someone was smoking medicinal pot prescribed by a doctor for glaucoma. Either way this is a poorly directed, scripted, acted and even produced (I never thought I'd sat that

In [ ]:
df = pd.DataFrame()
df['Reviews'] = reviews
df['Classifier'] = classifier
df

,Reviews,Classifier
0,"Solid comedy entertainment, with musical inter...",neg
1,The only good part about this film is the beau...,neg
2,From the beginning of the movie I had a feelin...,neg
3,"I have copy of this on VHS, I think they (The ...",neg
4,"I'm not a huge Freddy Krueger fan,but that doe...",neg
...,...,...
49995,I really enjoyed this movie. It was edgy witho...,pos
49996,One of the most magnificent movies ever made. ...,pos
49997,I found this flick enjoyable and involving to ...,pos
49998,The subject matter of this film is potentially...,pos


In [ ]:

## To split the data between test and train values
##
##
def split_data(percent=0.80):           #training data is 'percent' times of total
    values= np.array(df)
    np.random.shuffle(values)
    trainingDataLen = math.floor(len(values)*percent)
    #print (trainingDataLen)

    #print (data_given.iloc[trainingDataLen])
    trainingValues = values[:trainingDataLen, :]
    #testDataLen = len(self.values) - trainingDataLen
    testValues = values[trainingDataLen: , :]
    testDataLen = len(testValues)
    #print (testDataLen)
    return trainingValues,testValues
trainValues, testValues = split_data(0.75)  # 25pc Test and 75pc Train
print (np.shape(trainValues),np.shape(testValues))    

(37500, 2) (12500, 2)


In [ ]:
def remove_stopwords(text_tokens):
  #txt = txt.split()
  text_tokens = word_tokenize(text_tokens)
  tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
  return ' '.join(tokens_without_sw)
remove_stopwords(str(trainValues[1001,0]))  

"A brings new wife home former wife died `` accident '' . His new wife released institution VERY rich ! All sudden starts hearing noises seeing skulls place . Is going crazy first wife coming back dead ? < br / > < br / > You 've probably guessed ending I n't spell . I saw many times Saturday afternoon TV kid . Back , I liked I WAS young . Seeing I realize bad . It 's horribly acted , badly written , dull ( even hour ) huge cast FIVE people ( director ) ! Still good things . < br / > < br / > The music kinda creepy setting huge empty house pond nearby nicely atmospheric . There scary moments ( I jumped little saw first skull ) somewhat effective ending . All 's definitely NOT good movie ... total disaster either . It small cult following . I give 2. < br / > < br / > Also try avoid Elite DVD Drive-in edition ( 's paired `` Attack Giant Leeches '' ) . It 's TERRIBLE shape jumps scratches . It n't even look bad TV !"

In [ ]:

def tokenizer(txt):
  #f = open(txtfile,'r')
  #txtfile = f.read()
  txt = txt.translate(str.maketrans('', '', string.punctuation))#remove punctuation
  txtlist = word_tokenize(txt)
  txtlist_lower = [word.lower() for word in txtlist]
  counts = Counter(txtlist_lower)
  return counts


#function to count total frequency of a dictionary
def countTotalFreq(d):
  count = 0
  for i in d:
    count += d[i]
  return count  

def countNegativesPositives(classifier):
 
  classifier = list(classifier[:,1]) #positives and negatives
  counts = Counter(classifier)
  #print (counts)
  return counts




In [ ]:

class naiveBayes:

  def __init__(self,testData,trainData):
    self.testData = testData
    self.trainData = trainData
    #np.random.shuffle(trainData) 
    #np.random.shuffle(testData)
  
  def combineData(self,b=False):
    s = ""
    sPos =""
    sNeg =""
    if (b):
      for i in self.trainData:
        s+=i[0]
        if i[1] == 'neg':
          sNeg+=" "
          sNeg+=i[0] 
        elif i[1] == 'pos': 
          sPos+=" "
          sPos+=i[0]
      return tokenizer(s),tokenizer(sNeg),tokenizer(sPos)
    else:
      for i in self.testData:
        s+=i[0]
        if i[1] == 'neg':
          sNeg+=" "
          sNeg+=i[0] 
        elif i[1] == 'pos': 
          sPos+=" "
          sPos+=i[0] 
      return tokenizer(s),tokenizer(sNeg),tokenizer(sPos)
  
  def train(self):
    posnegCount = countNegativesPositives(self.trainData)  
    d = {}
    totalCount = posnegCount['pos'] + posnegCount['neg']
    #priors for both classes
    print (posnegCount['pos'],posnegCount['neg'])
    print ("length of posnegCount: ",len(posnegCount))
    priorsPos = np.log (float(posnegCount['pos']) / float(totalCount) )
    priorsNeg = np.log(float(posnegCount['neg']) / float(totalCount))
    
    #dictionaries containing all the
    # freqs and words in all the training documents
    combinedDict,negDict,posDict = self.combineData(True)
    print ("Combined all bro")

    # Probabilities
    probPos = {}
    probNeg = {}
    lenPosDict = countTotalFreq(posDict)
    lenNegDict = countTotalFreq(negDict)
    
    j = 0
    print ("|V| positive ",len(posDict))
    print ("count(positive) ", lenPosDict)
    print ("|V| negative ",len(negDict))
    print("count(negative) ", lenNegDict)
    for i in posDict:
      probPos[i] = np.log( (posDict[i] + 1.0) / (float(lenPosDict)  + float(len(posDict)) ))
      
      #probPos.append( posDict[i] + 1.0 / float(countTotalFreq(posDict))  + float(len(posDict)))
    print ("pos prob done")
    for i in negDict:
      probNeg[i] = np.log ((negDict[i] + 1.0) / ( float(lenNegDict) + len(negDict)   ) )
  

    return priorsPos,priorsNeg,probPos,probNeg
    #for w in combinedDict:
      #print(combinedDict[w])
    #return combinedDict  
    #print(len(combinedDict),len(negDict),len(posDict))
  def trainIt(self):
    self.priorsPos,self.priorsNeg,self.probPos,self.probNeg = self.train()
  
  def predict(self,txt):
    #priorsPos,priorsNeg,probPos,probNeg = self.train()
    txt = tokenizer(txt)
    pveProb = 1
    nveProb = 1
    #print (self.priorsNeg,self.priorsPos)
      #check if belongs to positive
    for i in txt:
      if(i in self.probPos):
        pveProb += ( self.priorsPos *  self.probPos[i]**txt[i] )
      if(i in self.probNeg):
        nveProb +=  ( self.priorsNeg * self.probNeg[i]**txt[i] )
    
    if (nveProb>pveProb):
      return "neg"
    else:
      return "pos"
  def evaluate(self):
    res = [self.predict(i[0]) for i in self.testData]
    num = float(len(self.testData[:,1] == res ))
    den = len(self.testData)
    return num/den  


myAlgo = naiveBayes(testValues,trainValues)
#combinedDict,negDict,posDict = myAlgo.combineData(True)
myAlgo.trainIt()
#print(super_dict)

18756 18744
length of posnegCount:  2
Combined all bro
|V| positive  104636
count(positive)  4360335
|V| negative  100182
count(negative)  4283439
pos prob done


In [ ]:
print(myAlgo.evaluate())
#myAlgo.predict("I did not like it but most people were favorable of this movie")

1.0
